In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import FL_utils.FL_interpretable_functions as interpret
import FL_utils.FL_RiskSLIM as slim
import FL_utils.FL_stumps as stumps
import FL_utils.FL_Corel as Corel

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\FL Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['misdemeanor_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['misdemeanor_two_year'].values

In [3]:
#### CART
depth = [1,2,3,4,5]
impurity = [0.001, 0.002,0.003]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, impurity, seed=816)

### EBM
estimators = [60,80,100]
depth = [1,2]
learning_rate = [0.01]
holdout_split = [0.7, 0.9]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, holdout_split, seed=816)

In [4]:
cart_summary['KY_score'], ebm_summary['KY_score']

(0.5737161485947689, 0.6447032090154788)

### Lasso Stumps

In [5]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['misdemeanor_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['misdemeanor_two_year'].values

## columns 
cols = KY_X.columns

In [7]:
single_stump_model = stumps.stump_model(FL_X, FL_Y, KY_X, KY_Y, 0.08, cols, 816)
single_stump_model['features']

['age_at_current_charge23.0',
 'age_at_current_charge24.0',
 'age_at_current_charge25.0',
 'age_at_current_charge29.0',
 'age_at_current_charge31.0',
 'age_at_current_charge62.0',
 'p_arrest4',
 'p_arrest6',
 'p_charges3',
 'p_violence3',
 'p_felony2',
 'p_misdemeanor2',
 'p_misdemeanor3',
 'p_property3',
 'p_felprop_viol1',
 'p_misdeassult1',
 'p_drug1',
 'p_fta_two_year1',
 'p_pending_charge1',
 'six_month1',
 'one_year1',
 'three_year1',
 'current_violence1']

In [8]:
c_grid={'C': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [9]:
stumps_summary['best_params'], stumps_summary['KY_score'], stumps_summary['auc_diffs']

({'C': 0.08}, 0.6201155248643363, 0.04552902182940899)

### RiskSLIM

In [10]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['misdemeanor_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['misdemeanor_two_year'].values

## columns 
cols = KY_X.columns

In [27]:
single_stump_model = stumps.stump_model(FL_X, 
                                        FL_Y, 
                                        KY_X, 
                                        KY_Y, 
                                        0.05, 
                                        cols, 816)
selected_features = single_stump_model['features']
len(selected_features)

14

In [28]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_FL_X.insert(0, '(Intercept)', 1)

In [29]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'misdemeanor_two_year', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000,
                                max_offset=100,
                                c=1e-6, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
11/12/19 @ 01:07 AM | 193 rows in lookup table
11/12/19 @ 01:07 AM | ------------------------------------------------------------
11/12/19 @ 01:07 AM | runnning initialization procedure
11/12/19 @ 01:07 AM | ------------------------------------------------------------
11/12/19 @ 01:07 AM | CPA produced 2 cuts
11/12/19 @ 01:07 AM | running naive rounding on 4 solutions
11/12/19 @ 01:07 AM | best objective value: 0.6931
11/12/19 @ 01:07 AM | rounding produced 4 integer solutions
11/12/19 @ 01:07 AM | best objective value is 0.6931
11/12/19 @ 01:07 AM | running sequential rounding on 4 solutions
11/12/19 @ 01:07 AM | best objective value: 0.6931
11/12/19 @ 01:07 AM | sequential rounding produced 0 integer solutions
11/12/19 @ 01:07 AM | polishing 4 solutions
11/12/19 @ 01:07 AM | best objective value: 0.6931
11/12/19 @ 01:07 AM | polishing produced 4 integer solutions
11/12/19 @ 01:07 AM | initialization produced 7 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 01:07 AM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5820.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 01:08 AM | adding 255 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5801.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 01:08 AM | adding 251 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5820.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 01:09 AM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.6005.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 01:10 AM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5831.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 01:10 AM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5855.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

In [30]:
riskslim_summary['KY_score'], np.mean(riskslim_summary['FL_validation_score'])

(0.6186985390560549, 0.575508380680412)

In [31]:
#### save results
summary_misdemeanor2_fl_inter_model = {"CART": cart_summary,
                                       "EBM": ebm_summary,
                                       "Stumps": stumps_summary,
                                       "RiskSLIM": riskslim_summary}

In [33]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\FL Model\\interpretable\\Two Year\\"
results = [["Misdemeanor",             
            round(cart_summary['KY_score'],3), 
            round(ebm_summary['KY_score'], 3), 
            round(stumps_summary['KY_score'],3), 
            round(riskslim_summary['KY_score'], 3)]]
with open(path + 'Two Year FL_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)